In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import datetime 
import os
from datetime import timedelta
from constants import *
from trip_file import TripFile


### Read all files

In [2]:
taarif = pd.read_csv("../files/taarif.csv")
taarif = taarif.drop(0).reset_index().drop(columns=["index"]) # first row is nulls
new_drivers = pd.read_csv("../files/new_drivers.csv", index_col=0)
drivers_with_kviut = pd.read_csv("../files/drivers_with_kviut.csv",index_col=0)

In [3]:
def preprocess_drivers(drivers: pd.DataFrame, copy=False) -> pd.DataFrame:
    df = drivers
    if copy:
        df = drivers.copy()

    gender_mapping = {"F": FEMALE,
                    "M":MALE,
                    "m":MALE,
                    "male":MALE,
                    "boy":MALE,
                    "unknown":UNKOWN,
                    'woman':FEMALE,
                    'girl':FEMALE,
                    'none':UNKOWN,
                    'female':FEMALE,
                    UNKOWN:UNKOWN}

    # Make preprocess
    df.gender = df.gender.fillna(UNKOWN)
    df.gender = df.gender.apply(lambda x: gender_mapping[x])
    df.birthdate = pd.to_datetime(df.birthdate, format="mixed")
    drivers.birthdate = drivers.birthdate.fillna(datetime.datetime.now())

    return df

def preprocess_taarif(taarif_df: pd.DataFrame, copy=False) -> pd.DataFrame:
    df = taarif_df
    if copy:
        df = new_drivers.copy()

    customers_mapping = {
            "dbs" : "yes",
            "mizranei_kfar_saba": "aminach",
            "telecommunication_ltd": "hot" 
    }

    for customer, new_customer in customers_mapping.items():
        # Select the row to duplicate (let's say the first row, index 0)
        row_to_duplicate = df[df["customer"] == customer]
        # Make a copy of the row
        new_row = row_to_duplicate.copy()
        new_row["customer"] = new_customer
        # Append the modified row to the DataFrame
        df = pd.concat([df,pd.DataFrame(new_row)], ignore_index=True)
        
    return df


# Concat the two tables of drivers with and without kviut
def concat_drivers_tables(new_drivers, drivers_with_kviut):
    drivers_with_kviut["kviut"] = 1
    drivers = pd.concat([new_drivers, drivers_with_kviut])
    drivers["kviut"] = drivers["kviut"].fillna(0)
    return drivers
    

In [4]:
drivers = concat_drivers_tables(new_drivers, drivers_with_kviut)
drivers = preprocess_drivers(drivers, copy=True)
taarif = preprocess_taarif(taarif)

In [44]:
from tqdm import tqdm
def get_trip_files(folder, limit=100):
    files = np.array(list(os.listdir(folder)))
    if limit:
        files = files[:limit]
    files = np.array([name if not name[-5] == ")" else name[:-7]+".csv" for name in files])
    print("Processing files....")
    return np.array([TripFile(os.path.join(folder,file)) for file in tqdm(files)])


def is_friday(time):
    return time.weekday() == 4
def is_after_4pm(time):
    return time.hour >=16
def is_saturday(time):
    return time.weekday() == 5
def is_before_8pm(time):
    return time.hour < 20 or (time.hour == 20 and time.minute == 0 and time.second == 0) 

def is_after_hour(time, hour):
    return time.hour >= hour
def is_before_hour(time, hour):
    return time.hour < hour or (time.hour == hour and time.minute == 0 and time.second == 0) 

def count_time_in_weekend(start_time, end_time):
    count = 0
    current_time = start_time

    while current_time < end_time:
        next_time = min(end_time, current_time + timedelta(hours=1))
        if is_friday(next_time):
            if is_after_hour(next_time, 16):
                friday_4pm = datetime.datetime(next_time.year, next_time.month, next_time.day, 16, 0)
                count += (next_time - max(current_time, friday_4pm)) / timedelta(hours=1)
        elif is_saturday(next_time):
            if is_before_hour(current_time, 20) or is_friday(current_time): # when next time jumps to midnight the current time is still on friday
                saturday_8pm= datetime.datetime(next_time.year, next_time.month, next_time.day, 20, 0)
                count += (min(next_time, saturday_8pm) - current_time) / timedelta(hours=1)
        
        current_time = current_time + timedelta(hours=1)

    return count

def count_drive_weekend_time(start_time, end_time):
    count = 0
    current_time = start_time

    while current_time < end_time:
        next_time = min(end_time, current_time + timedelta(days=1))
        if is_friday(current_time) or is_saturday(current_time):
            if is_friday(current_time):
                weekend_start = datetime.datetime(current_time.year, current_time.month, current_time.day, 22, 0)
                weekend_end = datetime.datetime(current_time.year, current_time.month, current_time.day, 16, 0)
            else:
                weekend_start = datetime.datetime(current_time.year, current_time.month, current_time.day, 22, 0)
                weekend_end = datetime.datetime(current_time.year, current_time.month, current_time.day, 22, 0)
            interval1 = (start_time, end_time)
            # interval2 = (night_start, night_end)
            # count += overlap_hours(interval1, interval2)

    return count



def count_time_at_night(start_time, end_time):
    count = 0
    current_time = start_time

    while current_time < end_time:
        next_time = min(end_time, current_time + timedelta(hours=1))
        if is_after_hour(next_time, 22):
                ten_pm = datetime.datetime(next_time.year, next_time.month, next_time.day, 22, 0)
                count += (next_time - max(current_time, ten_pm)) / timedelta(hours=1)
        elif is_before_hour(current_time, 6) or is_after_hour(current_time, 22):
                six_am = datetime.datetime(next_time.year, next_time.month, next_time.day, 6, 0)
                count += (min(next_time, six_am) - current_time) / timedelta(hours=1)
        
        current_time = current_time + datetime.timedelta(hours=1)
    return count


def count_drive_night_time(start_time, end_time):
    count = 0
    current_time = start_time

    while current_time < end_time:
        next_time = min(end_time, current_time + timedelta(days=1))
        night_start = datetime.datetime(current_time.year, current_time.month, current_time.day, 22, 0)
        night_end = night_start + timedelta(hours=8)
        interval1 = (start_time, end_time)
        interval2 = (night_start, night_end)
        count += overlap_hours(interval1, interval2)

    return count

        

def overlap_hours(interval1, interval2):
    start1, end1 = interval1
    start2, end2 = interval2

    # Check if intervals overlap
    if start1 <= end2 and start2 <= end1:
        # Calculate the overlap duration
        overlap_start = max(start1, start2)
        overlap_end = min(end1, end2)
        overlap_duration = overlap_end - overlap_start

        return overlap_duration.total_seconds() / 3600
    else:
        return 0


def count_time_at_night_and_weekend(start_time, end_time):
    count = 0
    current_time = start_time

    while current_time < end_time:
        next_time = min(end_time, current_time + timedelta(hours=1))
        if is_after_hour(next_time, 22):
            if is_friday(next_time):
                ten_pm = datetime.datetime(next_time.year, next_time.month, next_time.day, 22, 0)
                count += (next_time - max(current_time, ten_pm)) / timedelta(hours=1)
        elif (is_before_hour(current_time, 6) or is_after_hour(current_time, 22)):
            if is_saturday(next_time):
                six_am = datetime.datetime(next_time.year, next_time.month, next_time.day, 6, 0)
                count += (min(next_time, six_am) - current_time) / timedelta(hours=1)
        
        current_time = current_time + timedelta(hours=1)
    return count

# Calculate age function
def calculate_age(birth_date):
    current_date = datetime.datetime.now()
    if pd.isnull(birth_date):
        return np.nan
    age = current_date.year - birth_date.year - ((current_date.month, current_date.day) < (birth_date.month, birth_date.day))
    return age

def create_trips_df(trip_files):
    all_dfs = [trip_file.get_data_frame() for trip_file in trip_files ]
    trips_df = pd.concat(all_dfs)
    trips_df.end_time = pd.to_datetime(trips_df["end_time"])
    trips_df.start_time = pd.to_datetime(trips_df["start_time"])
    trips_df["kph"] = trips_df["km"] / ((trips_df["end_time"] - trips_df["start_time"]) / datetime.timedelta(hours=1))

    trips_df = trips_df.reset_index().drop(columns=["index"])
    trips_df["drive_time"] = (trips_df["end_time"]-trips_df["start_time"]) / timedelta(hours=1)

    sunday_morning = datetime.datetime(2024, 11, 5, 6, 0, 0) # sunday 6 am

    trips_df.loc[(trips_df["start_time"].isna()) & (trips_df["end_time"].isna()),"start_time"] = sunday_morning
    
    return trips_df
    

In [36]:
trip_files = get_trip_files("../files/trips_data", limit=1000)


Processing files....


  0%|          | 0/1000 [00:00<?, ?it/s]

100%|██████████| 1000/1000 [00:06<00:00, 165.27it/s]


In [50]:
trips_df = create_trips_df(trip_files)

In [31]:
from time import time
def calculate_trips_cost(trips_with_fares):
    
    start = time()
    df = trips_with_fares.copy()
    # print("copying time ", time() - start)
    
    start = time()
    
    night_and_weekend_hours =  df.apply(lambda r: count_time_at_night_and_weekend(r.start_time, r.end_time), axis=1)
    # print("calculate night and weekend time ", time() - start)
    start = time()

    weekend_hours = df.apply(lambda r: count_time_in_weekend(r.start_time, r.end_time), axis=1) - night_and_weekend_hours
    # print("calculate weekend time ", time() - start)
    start = time()

    # night_hours2 = df.apply(lambda r: count_drive_night_time(r.start_time, r.end_time), axis=1) - night_and_weekend_hours
    night_hours = df.apply(lambda r: count_time_at_night(r.start_time, r.end_time), axis=1) - night_and_weekend_hours
    # print("calculate night time ", time() - start)


    start = time()
    km_per_hour = df["km"] / ((df["end_time"]-df["start_time"]) / timedelta(hours=1))
    # print("calculate kph ", time() - start)
    
    start = time()
    basic_pay = df.km * df.basic_taarif
    extra_pay = df.km.apply(lambda km: max(0, km - 200)) * df.extra_milage
    payment_without_bonus = basic_pay + extra_pay
    payment_per_km = payment_without_bonus / df["km"]
    # print("calculate basic payments ", time() - start)


    start = time()
    # bonus_precentage = ((weekend_hours * df.weekend_bonus) + (night_hours * df.night_bonus) + (night_and_weekend_hours * (df.weekend_bonus + df.night_bonus)))
    night_extra_payment = (df.night_bonus / 100) * ((km_per_hour * night_hours) * payment_per_km)
    weekend_extra_payment = (df.weekend_bonus / 100) * ((km_per_hour * weekend_hours) * payment_per_km)
    night_and_weekend_extra_payment = ((df.weekend_bonus + df.night_bonus) / 100) * ((km_per_hour * night_and_weekend_hours) * payment_per_km)
    payment_with_bonus = payment_without_bonus + night_extra_payment + weekend_extra_payment + night_and_weekend_extra_payment
    # print("calculate bonuses ", time() - start)

    df["basic_pay"] = basic_pay
    df["extra_milage_pay"] = extra_pay
    df["weekend_hours"] = weekend_hours
    df["night_hours"] = night_hours
    df["night_and_weekend_hours"] = night_and_weekend_hours
    df["kph"] = km_per_hour
    df["weekend_km"] = weekend_hours * km_per_hour
    df["night_extra_payment"] = night_extra_payment
    df["weekend_extra_payment"] = weekend_extra_payment
    df["night_and_weekend_extra_payment"] = night_and_weekend_extra_payment
    df["bonus_payment"] = night_extra_payment + weekend_extra_payment + night_and_weekend_extra_payment
    df["payment_without_bonus"] = payment_without_bonus
    df["payment_with_bonus"] = payment_with_bonus
    
    return df


def create_main_table(trip_files=None, trips_df=None):
    cum_table = pd.DataFrame(columns=["driver_id","month","year","total_income","total_km"])
    if trip_files is None:
        trip_files = get_trip_files("../files/trips_data", limit=100)
    print("Processing income of drivers...")
    # for trip_file in tqdm(trip_files):
        
    #     trip_df = trip_file.get_data_frame()

    trips_with_fares = pd.merge(trips_df, taarif, on=["customer"],how="left")
    df = calculate_trips_cost(trips_with_fares)

    # print(df)
    subset_columns = ["driver_id", "month", "year", "payment_with_bonus", "km"]
    df_to_concat = df[subset_columns]
    new_column_names = {'payment_with_bonus': 'total_income', 'km': 'total_km'}
    df_to_concat = df_to_concat.rename(columns=new_column_names)
    
    cum_table = pd.concat([cum_table, df_to_concat], axis=0, join="outer")
    cum_table = cum_table.reset_index().drop(columns=["index"])
    cum_table = cum_table.groupby(["driver_id","month","year"]).sum().reset_index()

    drivers_copy = drivers.copy()
    drivers_copy["age"] = drivers.birthdate.apply(calculate_age)
    drivers_copy = drivers_copy.rename(columns={"id":"driver_id"})
        
    cum_table = pd.merge(cum_table, drivers_copy[["driver_id","gender","age","vetek","kviut"]], on=["driver_id"], how="left")

    return cum_table

table = create_main_table(trip_files, trips_df)

Processing income of drivers...


C:\Users\Ron Tohar\AppData\Local\Temp\ipykernel_13368\102371248.py:78: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  cum_table = pd.concat([cum_table, df_to_concat], axis=0, join="outer")


### Train model to predict missing times

In [72]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import FunctionTransformer
from sklearn.pipeline import make_pipeline
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

def preprocess_data_start_time(df1):
    df = df1.copy()
    df["drive_time"] = (df["end_time"]-df["start_time"]) / timedelta(hours=1)
    df["day_of_week"] = df["start_time"].dt.day.astype("category")
    df["is_weekend"] = df["day_of_week"].apply(lambda x: x==5 or x==6)
    df["hour"] = df["start_time"].dt.hour
    df["minute"] = df["start_time"].dt.minute
    return df[['day_of_week', 'hour', "minute","is_weekend", "km","kph"]]

def preprocess_data_end_time(df1):
    df = df1.copy()
    df["drive_time"] = (df["end_time"]-df["start_time"]) / timedelta(hours=1)
    df["day_of_week"] = df["end_time"].dt.day.astype("category")
    df["is_weekend"] = df["day_of_week"].apply(lambda x: x==5 or x==6)
    df["hour"] = df["end_time"].dt.hour
    df["minute"] = df["end_time"].dt.minute
    return df[['day_of_week', 'hour', "minute","is_weekend", "km","kph"]]

def train_model_start_time(trips_df):
    
    model = Pipeline([
        ("standard scaler", StandardScaler()),
        ('model', LinearRegression())  # Example model, replace with your own
    ])

    trips_with_times = trips_df[(~trips_df["start_time"].isna()) & (~trips_df["end_time"].isna())]

    data = preprocess_data_start_time(trips_with_times)
    X,y = data.drop(columns=["kph"]), data["kph"]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    model.fit(X_train, y_train)
    return model

def train_model_end_time(trips_df):
    
    model = Pipeline([
        ("standard scaler", StandardScaler()),
        ('model', LinearRegression())  # Example model, replace with your own
    ])

    trips_with_times = trips_df[(~trips_df["start_time"].isna()) & (~trips_df["end_time"].isna())]

    data = preprocess_data_end_time(trips_with_times)
    X,y = data.drop(columns=["kph"]), data["kph"]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    model.fit(X_train, y_train)
    return model

In [73]:


def get_slow_outliers_fixed(trips_df, model):
    outliers_slow = trips_df[(trips_df["drive_time"] >30) & ((trips_df["kph"] < 50)) ]
    outliers_slow_kph = model.predict(preprocess_data_start_time(outliers_slow).drop(columns="kph"))
    outliers_slow_fixed_drive_time = outliers_slow["km"] / outliers_slow_kph
    outliers_slow["fixed_drive_time"] = outliers_slow_fixed_drive_time
    outliers_slow["predicted_kph"] = outliers_slow_kph
    return outliers_slow

def get_fast_outliers_fixed(trips_df, model):
    outliers = trips_df[(trips_df["drive_time"] >30) & ((trips_df["kph"] > 150))]

    drivers_avg_km_df = trips_df[trips_df.driver_id.isin(outliers.driver_id.unique())][["driver_id", "km"]].groupby("driver_id").mean()
    drivers_avg_km_df = drivers_avg_km_df.rename(columns = {"km": "avg_km"})
    outliers = outliers.merge(drivers_avg_km_df, on=["driver_id"])

    outliers_kph = model.predict(preprocess_data_start_time(outliers).drop(columns="kph"))
    outliers_fixed_drive_time = outliers["avg_km"] / outliers_kph
    outliers["fixed_drive_time"] = outliers_fixed_drive_time
    outliers["predicted_kph"] = outliers_kph
    return outliers

def get_outliers_fixed(trips_df):
    model = train_model_start_time(trips_df)
    slow_outliers = get_slow_outliers_fixed(trips_df, model)
    fast_outliers = get_fast_outliers_fixed(trips_df,model)
    outliers_fixed = pd.concat([slow_outliers,fast_outliers])
    df = outliers_fixed
    df = pd.merge(trips_df, outliers_fixed[["driver_id", "customer","start_time","end_time","month","year","fixed_drive_time"]], on=["driver_id", "customer","start_time","end_time","month","year"], how="left") 
    df.loc[~df["fixed_drive_time"].isna(), "end_time"] = df[~df["fixed_drive_time"].isna()]["start_time"] + df[~df["fixed_drive_time"].isna()]["fixed_drive_time"].apply(lambda x: timedelta(hours=x)) # fix end time according to predicted drive time and start time
    return df

C:\Users\Ron Tohar\AppData\Local\Temp\ipykernel_13368\4199995009.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  outliers_slow["fixed_drive_time"] = outliers_slow_fixed_drive_time
C:\Users\Ron Tohar\AppData\Local\Temp\ipykernel_13368\4199995009.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  outliers_slow["predicted_kph"] = outliers_slow_kph


In [93]:
def get_missing_start_time_fixed(trips_df):
    model = train_model_end_time(trips_df)
    data = trips_df[trips_df["start_time"].isna()] # all rows without start time
    predicted_kph = model.predict(preprocess_data_end_time(data).drop(columns="kph"))
    fixed_drive_time = data["km"]/predicted_kph
    fixed_drive_timedelta = fixed_drive_time.apply(lambda x: timedelta(hours=x))
    trips_df.loc[trips_df["start_time"].isna(), "start_time"] = trips_df.loc[trips_df["start_time"].isna(), "end_time"] - fixed_drive_timedelta
    return trips_df

def get_missing_end_time_fixed(trips_df):
    model = train_model_start_time(trips_df)
    data = trips_df[trips_df["end_time"].isna()] # all rows without end time
    predicted_kph = model.predict(preprocess_data_start_time(data).drop(columns="kph"))
    fixed_drive_time = data["km"]/predicted_kph
    fixed_drive_timedelta = fixed_drive_time.apply(lambda x: timedelta(hours=x))
    trips_df.loc[trips_df["end_time"].isna(), "end_time"] = trips_df.loc[trips_df["end_time"].isna(), "start_time"] + fixed_drive_timedelta
    return trips_df

trips_df = create_trips_df(trip_files)
trips_df = get_outliers_fixed(trips_df)
trips_df = get_missing_start_time_fixed(trips_df)
trips_df = get_missing_end_time_fixed(trips_df)
# trips_df[trips_df["end_time"].isna()]

C:\Users\Ron Tohar\AppData\Local\Temp\ipykernel_13368\4199995009.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  outliers_slow["fixed_drive_time"] = outliers_slow_fixed_drive_time
C:\Users\Ron Tohar\AppData\Local\Temp\ipykernel_13368\4199995009.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  outliers_slow["predicted_kph"] = outliers_slow_kph


ValueError: Input y contains NaN.

,customer,driver_id,end_time,km,start_time,month,year,kph,drive_time
12833,yes,653,NaT,454.815943,NaT,april,2015,NaN,NaN
22709,yes,643,NaT,57.974586,NaT,april,2015,NaN,NaN
24041,osem,705,NaT,197.556857,NaT,april,2015,NaN,NaN
24580,intel,52,NaT,99.175149,NaT,april,2015,NaN,NaN
32551,tnuva,295,NaT,168.653256,NaT,august,2015,NaN,NaN
46365,tnuva,414,NaT,630.036986,NaT,august,2015,NaN,NaN
48658,jerusalem_muni,201,NaT,122.775826,NaT,august,2015,NaN,NaN
